<a href="https://colab.research.google.com/github/durml91/MMath-Project/blob/duo-branch/Image_Diffusion_(working)/Attention_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jaxlib==0.4.2+cuda11.cudnn82 -f  https://storage.googleapis.com/jax-releases/jax_cuda_releases.html # [cuda]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
    Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
      Successfully uninstalled jaxlib-0.3.25+cuda11.cudnn805


In [2]:
!pip install diffrax
!pip install equinox
!pip install einops
!pip install optax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jax: filename=jax-0.4.4-py3-none-any.whl size=1403844 sha256=fe15552e6a2b1f6f0b02a31657ed05dff3cb62d70053681fe7fe181c1015f93d
  Stored in directory: /root/.cache/pip/wheels/5e/cd/9b/750eb95db5b18b776ae59f55ae22b91a01e3703f3fb07eaa13
Successfully built jax
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1
  Attempting uninstall: jax
    Found existing installation: jax 0.3.25
    Uninstalling jax-0.3.25:
      Successfully uninstalled jax-0.3.25
Looking in indexes: https://pypi.org

In [3]:
import array
import functools as ft
import gzip
import os
import struct
import urllib.request

import diffrax as dfx  # https://github.com/patrick-kidger/diffrax
import einops  # https://github.com/arogozhnikov/einops
import jax
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
import optax  # https://github.com/deepmind/optax

import equinox as eqx

In [4]:
key = jr.PRNGKey(2023)

In [4]:
import shutil
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import shutil

### Original MLP Mixer + Attention

In [152]:
# class qkv(eqx.Module):
#     c_attn: eqx.nn.Linear


#     def __init__(self, key):
#         self.c_attn = eqx.nn.Linear(64, 3*64, key=key)

#     def __call__(self, x):
#         #x = einops.rearrange(x, "a b -> b a")
        
#         #T,C = x.shape
#         print("hi")
#         qkv = jax.vmap(self.c_attn)(x)
#         #qkv = qkv.transpose(1,0)
#         #print(qkv.shape)
#         q,k,v = jnp.array_split(qkv, 3, axis=-1)
#         print(q.shape,k.shape,v.shape)
#         # q = einops.rearrange(q, "a b->b a")
#         # k = einops.rearrange(k, "a b->b a")
#         # v = einops.rearrange(v, "a b->b a")

        
        
#         # #print(x.shape)
#         # W_q = self.wq(x)
#         # #print(W_q.shape)
#         # W_k = self.wk(x)
#         # W_v = self.wv(x)
#         # Q = jnp.matmul(x, W_q)
#         # K = jnp.matmul(x, W_k)
#         # V = jnp.matmul(x, W_v)

#         return q,k,v


# class Attblock(eqx.Module):
#     matrix: eqx.Module

#     c_proj: eqx.nn.Linear
    
#     att: eqx.nn.MultiheadAttention

#     def __init__(self, key):

#         self.att = eqx.nn.MultiheadAttention(7, query_size=64, key=key)
#         self.matrix = qkv(key=key)
#         self.c_proj = eqx.nn.Linear(64,64, key=key)

#     def __call__(self, x):
#         Q,K,V = self.matrix(x)
        
#         at=self.att(Q,K,V)
#         at = self.c_proj(at)
#         return at

In [130]:
# class qkv(eqx.Module):
#     c_attn: eqx.nn.Linear


#     def __init__(self, key):
#         self.c_attn = eqx.nn.Linear(66, 3*66, key=key)

#     def __call__(self, x):
#         x = einops.rearrange(x, "a b -> b a")
        
#         #T,C = x.shape

#         qkv = jax.vmap(self.c_attn)(x)
#         #qkv = qkv.transpose(1,0)
#         #print(qkv.shape)
#         q,k,v = jnp.array_split(qkv, 3, axis=-1)
#         print(q.shape,k.shape,v.shape)
        

        
        
#         # #print(x.shape)
#         # W_q = self.wq(x)
#         # #print(W_q.shape)
#         # W_k = self.wk(x)
#         # W_v = self.wv(x)
#         # Q = jnp.matmul(x, W_q)
#         # K = jnp.matmul(x, W_k)
#         # V = jnp.matmul(x, W_v)

#         return q,k,v


# class Attblock(eqx.Module):
#     matrix: eqx.Module

#     c_proj: eqx.nn.Linear
    
#     att: eqx.nn.MultiheadAttention

#     def __init__(self, key):

#         self.att = eqx.nn.MultiheadAttention(7, query_size=66, key=key)
#         self.matrix = qkv(key=key)
#         self.c_proj = eqx.nn.Linear(66,66, key=key)

#     def __call__(self, x):
#         Q,K,V = self.matrix(x)
        
#         at=self.att(Q,K,V)
#         at = self.c_proj(at)
#         return at



# class SinusoidalEmbedding(eqx.Module):
#     d:int = 28


#     def __init__(self, d):
#         self.d=d

#     def __call__(self, x):
#       half_dim = self.d//2
#       emb=jnp.log(10000) / (half_dim - 1)
#       emb = jnp.exp(jnp.arange(half_dim)* -emb)
#       emb = x[:,None] * emb[None,:]
#       emb = jnp.concatenate([jnp.sin(emb), jnp.cos(emb)], -1)
#       return emb


# class TimeEmbed(eqx.Module):
#     dim: int =28
#     embed: eqx.nn.Embedding
#     sin: eqx.Module
    
#     def __init__(self, key):
#         self.embed = eqx.nn.Embedding(1000, 28, key=key)
#         self.sin = SinusoidalEmbedding(28)
  
#     def __call__(self, x):
#         se = self.sin(x)
#         se = (se*10000).astype(int)
#         emb = self.embed(se)
#         return emb




# class MixerBlock(eqx.Module):
#     patch_mixer: eqx.nn.MLP
#     hidden_mixer: eqx.nn.MLP
#     norm1: eqx.nn.LayerNorm
#     norm2: eqx.nn.LayerNorm

#     Att1: eqx.Module
#     Att2: eqx.Module
#     gnorm1: eqx.nn.GroupNorm
#     gnorm2: eqx.nn.GroupNorm


#     def __init__(
#         self, num_patches, hidden_size, mix_patch_size, mix_hidden_size, *, key
#     ):
#         tkey, ckey,a1key, a2key = jr.split(key, 4)
#         self.patch_mixer = eqx.nn.MLP(
#             num_patches, num_patches, mix_patch_size, depth=1, key=tkey
#         )
#         self.hidden_mixer = eqx.nn.MLP(
#             hidden_size, hidden_size, mix_hidden_size, depth=1, key=ckey
#         )
#         self.norm1 = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.norm2 = eqx.nn.LayerNorm((num_patches, hidden_size))

#         self.Att1 = Attblock(key = a1key)
#         self.Att2 = Attblock(key = a2key)

#         self.gnorm1 = eqx.nn.GroupNorm(4,32)
#         self.gnorm2 = eqx.nn.GroupNorm(4,32)

#     def __call__(self, y):
#         y = y + jax.vmap(self.patch_mixer)(self.norm1(y))
#         unatt = self.Att1(y)
#         gnormatt = self.gnorm1(unatt)
#         y = y + gnormatt
        
#         y = einops.rearrange(y, "c p -> p c")
#         y = y + jax.vmap(self.hidden_mixer)(self.norm2(y))
#         y = einops.rearrange(y, "p c -> c p")
        
#         unatt = self.Att2(y)
#         gnormatt = self.gnorm2(unatt)
#         y = y + gnormatt
#         return y




# class Mixer2d(eqx.Module):
#     conv_in: eqx.nn.Conv2d
#     conv_out: eqx.nn.ConvTranspose2d
#     blocks: list
#     norm: eqx.nn.LayerNorm
#     t1: float
#     timemb: eqx.Module


#     def __init__(
#         self,
#         img_size,
#         patch_size,
#         hidden_size,
#         mix_patch_size,
#         mix_hidden_size,
#         num_blocks,
#         t1,
#         *,
#         key,
#     ):
#         input_size, height, width = img_size
#         assert (height % patch_size) == 0
#         assert (width % patch_size) == 0
#         num_patches = (height // patch_size) * (width // patch_size)
        
#         inkey, outkey,tkey, *bkeys = jr.split(key, 3 + num_blocks)

#         self.conv_in = eqx.nn.Conv2d(
#             input_size + 1, hidden_size, patch_size, stride=patch_size, key=inkey
#         )
#         self.conv_out = eqx.nn.ConvTranspose2d(
#             hidden_size, input_size, patch_size, stride=patch_size, key=outkey
#         )
#         self.blocks = [
#             MixerBlock(
#                 num_patches, hidden_size, mix_patch_size, mix_hidden_size, key=bkey
#             )
#             for bkey in bkeys
#         ]
#         self.norm = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.t1 = t1
#         self.timemb = TimeEmbed(key=tkey)

#     def __call__(self, t, y):
#         t = t / self.t1
#         t = einops.repeat(t, "-> 1")
#         t = jnp.array([t], dtype=int)
#         t = self.timemb(t)
#         t = einops.rearrange(t, " c 1 h w-> c h w")
#         y = jnp.concatenate([y, t])
#         y = self.conv_in(y)
#         _, patch_height, patch_width = y.shape
#         y = einops.rearrange(y, "c h w -> c (h w)")
#         for block in self.blocks:
#             y = block(y)
#         y = self.norm(y)
#         y = einops.rearrange(y, "c (h w) -> c h w", h=patch_height, w=patch_width)
#         return self.conv_out(y)

In [34]:
# class qkv(eqx.Module):
#     wk: eqx.nn.Linear
#     wq: eqx.nn.Linear
#     wv: eqx.nn.Linear

#     def __init__(self, key):
#         self.wq = eqx.nn.Linear(49, 128, key=key)
#         self.wk = eqx.nn.Linear(49, 128, key=key)
#         self.wv = eqx.nn.Linear(49, 128, key=key)

#     def __call__(self, x):
#         W_q = jax.vmap(self.wq)(x)
#         W_k = jax.vmap(self.wk)(x)
#         W_v = jax.vmap(self.wv)(x)
        
#         print(W_q.shape, x.shape)
#         Q = jax.lax.batch_matmul(x, W_q)
#         K = jax.lax.batch_matmul(x, W_k)
#         V = jax.lax.batch_matmul(x, W_v)
#         return Q,K,V


# class Attblock(eqx.Module):
#     matrix: eqx.Module
    
#     att: eqx.nn.MultiheadAttention

#     def __init__(self, key):

#         self.att = eqx.nn.MultiheadAttention(7, query_size=49, key=key)
#         self.matrix = qkv(key=key)

#     def __call__(self, x):
#         Q,K,V = self.matrix(x)
        
#         at=self.att(Q,K,V)
#         return at




# class MixerBlock(eqx.Module):
#     patch_mixer: eqx.nn.MLP
#     hidden_mixer: eqx.nn.MLP
#     norm1: eqx.nn.LayerNorm
#     norm2: eqx.nn.LayerNorm

#     Att1: eqx.Module
#     Att2: eqx.Module
#     gnorm1: eqx.nn.GroupNorm
#     gnorm2: eqx.nn.GroupNorm


#     def __init__(
#         self, num_patches, hidden_size, mix_patch_size, mix_hidden_size, *, key
#     ):
#         tkey, ckey,a1key, a2key = jr.split(key, 4)
#         self.patch_mixer = eqx.nn.MLP(
#             num_patches, num_patches, mix_patch_size, depth=1, key=tkey
#         )
#         self.hidden_mixer = eqx.nn.MLP(
#             hidden_size, hidden_size, mix_hidden_size, depth=1, key=ckey
#         )
#         self.norm1 = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.norm2 = eqx.nn.LayerNorm((num_patches, hidden_size))

#         self.Att1 = Attblock(key = a1key)
#         self.Att2 = Attblock(key = a2key)

#         self.gnorm1 = eqx.nn.GroupNorm(4,32)
#         self.gnorm2 = eqx.nn.GroupNorm(4,32)

#     def __call__(self, y):
#         y = y + jax.vmap(self.patch_mixer)(self.norm1(y))
#         unatt = self.Att1(y)
#         gnormatt = self.gnorm1(unatt)
#         y = y + gnormatt
        
#         y = einops.rearrange(y, "c p -> p c")
#         y = y + jax.vmap(self.hidden_mixer)(self.norm2(y))
#         y = einops.rearrange(y, "p c -> c p")
        
#         unatt = self.Att2(y)
#         gnormatt = self.gnorm2(unatt)
#         y = y + gnormatt
#         return y




# class Mixer2d(eqx.Module):
#     conv_in: eqx.nn.Conv2d
#     conv_out: eqx.nn.ConvTranspose2d
#     blocks: list
#     norm: eqx.nn.LayerNorm
#     t1: float
#     tim_emb: eqx.nn.Embedding


#     def __init__(
#         self,
#         img_size,
#         patch_size,
#         hidden_size,
#         mix_patch_size,
#         mix_hidden_size,
#         num_blocks,
#         t1,
#         *,
#         key,
#     ):
#         input_size, height, width = img_size
#         assert (height % patch_size) == 0
#         assert (width % patch_size) == 0
#         num_patches = (height // patch_size) * (width // patch_size)
        
#         inkey, outkey,tkey, *bkeys = jr.split(key, 3 + num_blocks)

#         self.conv_in = eqx.nn.Conv2d(
#             input_size + 1, hidden_size, patch_size, stride=patch_size, key=inkey
#         )
#         self.conv_out = eqx.nn.ConvTranspose2d(
#             hidden_size, input_size, patch_size, stride=patch_size, key=outkey
#         )
#         self.blocks = [
#             MixerBlock(
#                 num_patches, hidden_size, mix_patch_size, mix_hidden_size, key=bkey
#             )
#             for bkey in bkeys
#         ]
#         self.norm = eqx.nn.LayerNorm((hidden_size, num_patches))
#         self.t1 = t1
#         self.tim_emb = eqx.nn.Embedding(28, 28, key=tkey)

#     def __call__(self, t, y):
#         t = t / self.t1
#         t = einops.repeat(t, "-> 1")
#         t = jnp.array([t], dtype=int)
#         t = self.tim_emb(t)
        
#         #_, height, width = y.shape
#         t = einops.repeat(t, "c h w-> c (28 h) w")
#         y = jnp.concatenate([y, t])
#         y = self.conv_in(y)
#         _, patch_height, patch_width = y.shape
#         y = einops.rearrange(y, "c h w -> c (h w)")
#         for block in self.blocks:
#             y = block(y)
#         y = self.norm(y)
#         y = einops.rearrange(y, "c (h w) -> c h w", h=patch_height, w=patch_width)
#         return self.conv_out(y)

### DiT Model

##### Time embedding

In [5]:
from typing import Callable

In [6]:
#Embedding Layer of Timesteps

class Lambda1(eqx.Module):
    fn: Callable

    def __call__(self , x, *, key=None):
        return self.fn(x)



class TimeStepEmbedder(eqx.Module):
    mlp: eqx.nn.Sequential
    frequency_embedding_size: int
    
    
    def __init__(
        self,
        hidden_size,
        frequency_embedding_size,   #set as 256
        key
    ):

        self.mlp = eqx.nn.Sequential([
            eqx.nn.Linear(frequency_embedding_size, hidden_size, key=key),
            Lambda1(jax.nn.silu),
            eqx.nn.Linear(hidden_size, hidden_size, key=key)
        ])
        self.frequency_embedding_size = frequency_embedding_size


    def __call__(
        self,
        t,
        max_period=10000
    ):
        dim = self.frequency_embedding_size
        half = dim // 2
        freqs = jnp.exp(
            -jnp.log(max_period) * jnp.arange(0, half, dtype=float) / half
        )
        args = t[:, None].astype(float) * freqs[None]
        embedding = jnp.concatenate([jnp.cos(args), jnp.sin(args)], axis=-1)
        if dim % 2:
            embedding = jnp.concatenate([embedding, jnp.zeros_like(embedding[:, :1])], axis=-1)
        t_freq = embedding
        t_emb = jax.vmap(self.mlp)(t_freq)
        return t_emb      

In [7]:
#Time embed test
TimeStepEmbedder(32, 256, key)(jnp.array([4]))

Array([[ 0.3052112 ,  0.19230524,  0.14764252,  0.31935525, -0.06504469,
        -0.23124412, -0.36936757,  0.02332152,  0.14454725, -0.18957621,
         0.20766026,  0.00606484,  0.03791146, -0.04720105, -0.12594688,
        -0.15844625, -0.09823439,  0.18520194, -0.12404346, -0.09638146,
        -0.04518805,  0.06181648, -0.0097344 , -0.07609019, -0.00168262,
         0.21857136, -0.17319855,  0.19038439, -0.12791844, -0.2365416 ,
         0.14090869,  0.08538287]], dtype=float32)

##### Multihead Attention

In [286]:
## attention

class MultiAtt(eqx.Module):
    c_attn: eqx.nn.Linear
    c_proj: eqx.nn.Linear
    n_head: int

    def __init__(
        self,
        n_embd,
        n_head,
        key
    ):
        assert n_embd % n_head ==0
        self.c_attn = eqx.nn.Linear(n_embd, 3 * n_embd, key=key)
        self.c_proj = eqx.nn.Linear(n_embd, n_embd, key=key)
        self.n_head = n_head
        #self.mha = eqx.nn.MultiheadAttention(n_head, )

    def __call__(
        self,
        x,
        mask=None
    ):
        B, T, C = x.shape
        qkv = jax.vmap(jax.vmap(self.c_attn))(x)
        qkv = qkv.reshape(B, T, self.n_head, -1)
        qkv = jnp.transpose(qkv, (0, 2, 1, 3)) #[batch, number of heads, T or seq_length, embed_dim or C(channels)]
        q, k, v = jnp.array_split(qkv, 3, axis=-1)
        new_k = jnp.swapaxes(k, -2,-1) *(1.0 / jnp.sqrt(k.shape[-1]))
        att = jax.lax.batch_matmul(q, new_k)
        if mask is not None:
            attn = jnp.where(mask == 0, -9e15, att)
        attention = jax.nn.softmax(att, axis=-1)
        values = jax.lax.batch_matmul(attention, v)
        values = jnp.transpose(values, (0,2,1,3)) # [batch, T, n_heads, C]
        values = values.reshape(B, T, C)
        o = jax.vmap(jax.vmap(self.c_proj))(values)
        return o   #, attention

In [9]:
x = jr.normal(key, (3, 16, 128))
print(x.shape)
mh_attn = MultiAtt(n_embd=128, n_head=4, key=key)
output, attention = mh_attn(x)
print(output.shape, attention.shape)

(3, 16, 128)
(3, 16, 128) (3, 4, 16, 16)


In [ ]:
# ### Extra misc

# class Lambda2(eqx.Module):
#     fn: Callable

#     def __call__(self , x, *, key=None):
#         return self.fn(x)

##### Utility function -modulate

In [290]:
def modulate(x, shift, scale):
    scale = jnp.expand_dims(scale, axis=0)   #"a -> a 1"
    shift = jnp.expand_dims(shift, axis=0)      #"a -> a 1"
    
    #return x * (1 + scale) + shift
    c = (1 + scale) + shift
    #print(x.shape, c.shape)
    return x * c
    #print(x.shape, c.shape)
    #return jnp.matmul(x,c)

In [202]:
#test
w = jr.normal(key, (4,8,32))
q=jnp.arange(0,8)
p=jnp.arange(0,8)
modulate(w, q, p)

##### DiT block

In [313]:
## DitBlock

class DitBlock(eqx.Module):
    norm1: eqx.nn.LayerNorm
    norm2: eqx.nn.LayerNorm
    attn: eqx.Module
    Mlp: eqx.nn.MLP
    adaLN_modulation: eqx.nn.Sequential


    def __init__(
        self,
        hidden_size,
        n_head,
        mlp_ratio,   # = 4.0
        key,
    ):
        self.norm1 = eqx.nn.LayerNorm(hidden_size, eps = 1e-06, elementwise_affine=False)
        self.attn = MultiAtt(hidden_size, n_head=n_head, key=key)
        self.norm2 = eqx.nn.LayerNorm(hidden_size, eps = 1e-06, elementwise_affine=False)
        mlp_hidden_size = int(hidden_size * mlp_ratio)
        self.Mlp = eqx.nn.MLP(hidden_size, hidden_size, mlp_hidden_size, 1, key=key)
        self.adaLN_modulation = eqx.nn.Sequential([
            Lambda1(jax.nn.silu),
            eqx.nn.Linear(hidden_size, 6 * hidden_size, key=key)
        ])

    def __call__(
        self,
        x,
        c
    ):
        temp = jax.vmap(self.adaLN_modulation)(c)
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = jnp.array_split(temp, 6, axis=1)
        #gate_msa = einops.rearrange(gate_msa, "a -> a 1")
        #print(gate_msa.shape)

        gate_msa = jnp.expand_dims(gate_msa, axis=0)
        #print(gate_msa.shape)

        a = jax.vmap(self.norm1)(x)
        #print(a.shape, shift_msa.shape)
      
        #print(a.shape, shift_msa.shape, scale_msa.shape)
        tem = modulate(a, shift_msa, scale_msa)
        #print(tem.shape)

        x = x + gate_msa * self.attn(tem)
        #gate_mlp = einops.rearrange(gate_mlp, "a -> a 1")

        gate_mlp = jnp.expand_dims(gate_mlp, axis=0)
        b = jax.vmap(self.norm2)(x)
        tems = modulate(self.norm2(x), shift_mlp, scale_mlp)     
        x = x + gate_mlp * jax.vmap(jax.vmap(self.Mlp))(tems)
        return x


In [ ]:
DitBlock(128, 16, 4.0, key)

In [ ]:
jr.normal(key, (2,2,2))

In [ ]:
# eqx.nn.MLP(5,16, 20, 3, jax.nn.gelu, key=key)

##### Final Layer

In [349]:
## FinalLayer

class FinalLayer(eqx.Module):
    norm_final: eqx.nn.LayerNorm
    linear: eqx.nn.Linear
    adaLN_modulation: eqx.nn.Linear    #eqx.nn.Sequential
    
    
    def __init__(
        self,
        hidden_size,
        patch_size,
        out_channels,
        key
    ):

        self.norm_final = eqx.nn.LayerNorm(hidden_size, eps=1e-6, elementwise_affine=False)
        self.linear = eqx.nn.Linear(hidden_size, patch_size * patch_size * out_channels, key=key)
        # self.adaLN_modulation = eqx.nn.Sequential([
        #     Lambda1(jax.nn.silu),
        #     eqx.nn.Linear(hidden_size, 2 * hidden_size, key=key)
        # ])

        self.adaLN_modulation = eqx.nn.Linear(hidden_size, 2 * hidden_size, key=key)

    def __call__(
        self,
        x,
        c
    ):
        
        c = jax.nn.silu(c)
        
        temp = jax.vmap(self.adaLN_modulation)(c)
        shift, scale = jnp.array_split(temp, 2, axis=1)
        x = modulate(self.norm_final(x), shift, scale)
        x = jax.vmap(jax.vmap(self.linear))(x)
        return x

In [ ]:
FinalLayer(16,2,2,key=key)

##### PatchEmbed

In [403]:
class PatchEmbed(eqx.Module):
    num_patches:int
    proj: eqx.nn.Conv2d
    patch_size: int


    def __init__(
        self,
        img_size,
        patch_size,
        in_chans,
        n_embd,
        key
    ):
        self.patch_size = patch_size
        dg = img_size // self.patch_size
        self.num_patches = dg ** 2
        self.proj = eqx.nn.Conv2d(in_chans, n_embd, self.patch_size, self.patch_size, key=key)

    def __call__(
        self,
        x
    ):
        B, C, H, W = x.shape
        x = jnp.array(x, dtype=float)
        x = jax.vmap(self.proj)(x)
        x = einops.rearrange(x, "B C H W -> B (H W) C")
        return x

In [149]:
eg = eg.astype(float)

In [ ]:
PatchEmbed(28, 4, 1, 384, key=key)(eg)

##### Utility functions - positional embedding functions

In [77]:
def get_2d_sincos_pos_embed(n_embd, grid_size):

    grid_h = jnp.arange(grid_size, dtype=float)
    grid_w = jnp.arange(grid_size, dtype=float)
    grid = jnp.meshgrid(grid_w, grid_h)
    grid = jnp.stack(grid, axis=0)

    #grid = einops.rearrange(grid, " 2 d f -> 2 1 d f")
    grid = jnp.reshape(grid, (2, 1, grid_size, grid_size))
    pos_embed = get_2d_sincos_pos_embed_from_grid(n_embd, grid)
    return pos_embed

In [78]:
g = get_2d_sincos_pos_embed(16, 8)

In [79]:
g.shape

(64, 16)

In [70]:
def get_2d_sincos_pos_embed_from_grid(n_embd, grid):
    assert n_embd % 2 == 0


    emb_h = get_1d_sincos_pos_embed_from_grid(n_embd // 2, grid[0]) # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(n_embd // 2, grid[1]) # (H*W, D/2)

    emb = jnp.concatenate([emb_h, emb_w], axis=1)  #(H*W, D)
    return emb

In [59]:
b = jr.normal(key, (3,3))

In [69]:
c = get_2d_sincos_pos_embed_from_grid(16, b)

(3,)


In [65]:
c.shape

(3, 16)

In [63]:
def get_1d_sincos_pos_embed_from_grid(n_embd, pos):
    assert n_embd % 2 == 0
    omega = jnp.arange(n_embd // 2, dtype=float)
    omega /= n_embd / 2
    omega = 1. / 10000*omega #(D/2)

    pos = jnp.array(pos)
    #print(pos.shape)
    #pos = einops.rearrange(pos, "a -> a 0")
    out = jnp.outer(pos, omega)

    emb_sin = jnp.sin(out)
    emb_cos = jnp.cos(out)

    emb = jnp.concatenate([emb_sin, emb_cos], axis=1)
    return emb

In [57]:
a=get_1d_sincos_pos_embed_from_grid(32, [1,2,3,4])

(4,)


In [58]:
a.shape

(4, 32)

##### Parameter class

In [81]:
# from typing import NamedTuple

In [89]:
class Params(eqx.Module):
    param: jnp.ndarray

    def __init__(self, num_patches, hidden_size):
        self.param = jnp.zeros((1, num_patches, hidden_size), dtype = float)

    def __call__(self):
        return self.param

In [90]:
a = Params(8, 12)()

In [92]:
a.shape

(1, 8, 12)

##### DiT

In [433]:
class DiT(eqx.Module):
    in_channels: int
    out_channels: int
    patch_size: int
    n_head: int
    
    x_embedder: eqx.Module
    t_embedder: eqx.Module
    pos_embed: eqx.Module
    blocks: list
    final_layer: eqx.Module
    conv_out: eqx.nn.ConvTranspose2d

    def __init__(
        self,
        input_size=28,
        patch_size=4,
        in_channels=1,
        hidden_size=384,
        depth=4,
        n_head=6,
        mlp_ratio=4.0,
        key=key
    ):
        self.in_channels = in_channels
        self.out_channels = in_channels
        self.patch_size = patch_size
        self.n_head = n_head


        self.x_embedder = PatchEmbed(input_size, patch_size, in_channels, hidden_size, key=key)
        self.t_embedder = TimeStepEmbedder(hidden_size, frequency_embedding_size=256, key=key)
        num_patches = self.x_embedder.num_patches


        self.pos_embed = Params(num_patches, hidden_size)

        #*bkeys = jr.split(key, num_blocks)

        self.blocks = [
            DitBlock(
                hidden_size, n_head, mlp_ratio, key = key
            )
            for _ in range(depth)
        ]

        self.final_layer = FinalLayer(hidden_size, patch_size, self.out_channels, key=key)
        

        self.conv_out = eqx.nn.ConvTranspose2d(49, input_size, patch_size, patch_size, key=key)

    
    def unpatchify(self, x):
        """
        x: (N, T, patch_size ** 2 * C)
        imgs: (N, H, W, C)
        """
        c = self.out_channels      
        p = self.x_embedder.patch_size 
        h = w = int(x.shape[1] ** 0.5)    

        print(h, x.shape)
        assert h * 2 == x.shape

        x = jnp.reshape(x, (x.shape[0], h, w, p, p, c))
        x = einops.rearrange(x, "n h w p q c->n c h p w q")
        imgs = jnp.reshape(x, (x.shape[0], c, h * p, h * p))
        return imgs
    
    def __call__(self, x, t):
        pos_embed = get_2d_sincos_pos_embed(self.pos_embed().shape[-1], int(self.x_embedder.num_patches ** 0.5))
        
        """
        x: (N, C, H, W)
        t: (N, )
        """
        t = jnp.array(t, dtype=int)
        x = self.x_embedder(x) + pos_embed  # (N, T, D), where T = H * W / patch_size ** 2
        t = self.t_embedder(t)   # (N, D)
        for block in self.blocks:
            x = block(x,t)    # (N, T, D)

        print(x.shape)
        x = self.final_layer(x, t)     # (N, T, patch_size ** 2 * out_channels) - N is the batch_size, T is the number of patches, 
        print(x.shape)
        x = einops.rearrange(x, " n c (h w) -> n c h w", h=4, w=4)


        #x = self.unpatchify(x)        # (N, out_channels, H, W)

        x = jax.vmap(self.conv_out)(x)


        #x = jnp.squeeze(x, axis=0)
        return x


##### Test

In [434]:
model = DiT()

Test

In [381]:
data = mnist()

In [382]:
data.shape

(60000, 1, 28, 28)

In [383]:
eg = data[0]
eg = eg[None,:,:,:]

In [384]:
eg.shape

(1, 1, 28, 28)

In [385]:
t = jnp.array([2], dtype = float)

In [435]:
model(eg, t)

(1, 49, 384)
(1, 49, 16)


Array([[[[-0.15487967, -0.03433783,  0.12822291, ..., -0.09030321,
           0.03372749,  0.00213137],
         [ 0.06128251, -0.07565106, -0.02559442, ..., -0.03945857,
           0.04431188,  0.04640357],
         [ 0.19963561, -0.03064599, -0.15983671, ...,  0.00627354,
           0.03675549,  0.1321956 ],
         ...,
         [ 0.13916822,  0.06520283,  0.04045987, ...,  0.03250512,
           0.16485381, -0.14930223],
         [ 0.095825  ,  0.03038388,  0.03251266, ..., -0.21594419,
           0.11069892,  0.17600389],
         [-0.03835377, -0.02780291, -0.02675361, ...,  0.08678256,
           0.21682394,  0.0822638 ]],

        [[-0.20123295, -0.02842518, -0.09395275, ..., -0.14093553,
           0.06253438, -0.00186716],
         [-0.09244944, -0.19457988,  0.01205789, ...,  0.13205373,
           0.00641725, -0.0593353 ],
         [ 0.00585786,  0.07106563,  0.05164475, ...,  0.09604876,
           0.01921992, -0.03600349],
         ...,
         [ 0.02757796, -0.09348467

##### Misc

In [172]:
import torch

In [247]:
B, M, T = 1, 49, 384

In [261]:
a = torch.rand((1, M, T))
b = torch.rand((1, T))

In [262]:
def mod(x, shift, scale):
    print(scale.unsqueeze(1).shape)
    print(x.shape)
    c = (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)
    print(c.shape)
    print((x * c).shape)
    return x * c

In [263]:
er = mod(a , b, b)

torch.Size([1, 1, 384])
torch.Size([1, 49, 384])
torch.Size([1, 1, 384])
torch.Size([1, 49, 384])


In [ ]:
#   embed_dim = n_embd
#   num_heads = n_head

#   don't forget to Jit utility functions

In [124]:
from itertools import repeat
import collections.abc
import torch

In [125]:

# From PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable) and not isinstance(x, str):
            return tuple(x)
        return tuple(repeat(x, n))
    return parse

In [129]:
_ntuple(2)(16)[0]

16

### Rest of code

In [372]:
def single_loss_fn(model, weight, int_beta, data, t, key):
    mean = data * jnp.exp(-0.5 * int_beta(t))
    var = jnp.maximum(1 - jnp.exp(-int_beta(t)), 1e-5)
    std = jnp.sqrt(var)
    noise = jr.normal(key, data.shape)
    y = mean + std * noise
    y = jnp.expand_dims(y, axis=0)
    pred = model(y,t)
    return weight(t) * jnp.mean((pred + noise / std) ** 2)


def batch_loss_fn(model, weight, int_beta, data, t1, key):
    batch_size = data.shape[0]
    tkey, losskey = jr.split(key)
    losskey = jr.split(losskey, batch_size)
    # Low-discrepancy sampling over t to reduce variance
    t = jr.uniform(tkey, (batch_size,), minval=0, maxval=t1 / batch_size)
    t = t + (t1 / batch_size) * jnp.arange(batch_size)
    loss_fn = ft.partial(single_loss_fn, model, weight, int_beta)
    loss_fn = jax.vmap(loss_fn)
    return jnp.mean(loss_fn(data, t, losskey))

In [93]:
def mnist():
    filename = "train-images-idx3-ubyte.gz"
    url_dir = "https://storage.googleapis.com/cvdf-datasets/mnist"
    target_dir = os.getcwd() + "/data/mnist"
    url = f"{url_dir}/{filename}"
    target = f"{target_dir}/{filename}"

    if not os.path.exists(target):
        os.makedirs(target_dir, exist_ok=True)
        urllib.request.urlretrieve(url, target)
        print(f"Downloaded {url} to {target}")

    with gzip.open(target, "rb") as fh:
        _, batch, rows, cols = struct.unpack(">IIII", fh.read(16))
        shape = (batch, 1, rows, cols)
        return jnp.array(array.array("B", fh.read()), dtype=jnp.uint8).reshape(shape)


def dataloader(data, batch_size, *, key):
    dataset_size = data.shape[0]
    indices = jnp.arange(dataset_size)
    while True:
        perm = jr.permutation(key, indices)
        (key,) = jr.split(key, 1)
        start = 0
        end = batch_size
        while end < dataset_size:
            batch_perm = perm[start:end]
            yield data[batch_perm]
            start = end
            end = start + batch_size
     

In [355]:
@eqx.filter_jit
def make_step(model, weight, int_beta, data, t1, key, opt_state, opt_update):
    loss_fn = eqx.filter_value_and_grad(batch_loss_fn)
    loss, grads = loss_fn(model, weight, int_beta, data, t1, key)
    updates, opt_state = opt_update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    key = jr.split(key, 1)[0]
    return loss, model, key, opt_state

In [357]:
t1=10.0
# Optimisation hyperparameters
num_steps=500_000
lr=3e-4
batch_size=256
print_every=5_000

In [376]:
data = mnist()
data_mean = jnp.mean(data)
data_std = jnp.std(data)
data_max = jnp.max(data)
data_min = jnp.min(data)
data_shape = data.shape[1:]
data = (data - data_mean) / data_std

In [359]:
loader_key, train_key, model_key = jr.split(key, 3)

In [377]:
model = DiT(key = model_key)

In [361]:
int_beta = lambda t: t  # Try experimenting with other options here!
weight = lambda t: 1 - jnp.exp(-int_beta(t))  

In [378]:
opt = optax.adabelief(lr)
# Optax will update the floating-point JAX arrays in the model.
opt_state = opt.init(eqx.filter(model, eqx.is_inexact_array))

In [379]:
total_value = 0
total_size = 0
for step, data in zip(range(num_steps), dataloader(data, batch_size, key=loader_key)): 
    value, model, train_key, opt_state = make_step(model, weight, int_beta, data, t1, train_key, opt_state, opt.update)
    total_value += value.item()
    total_size += 1
    if (step % print_every) == 0 or step == num_steps - 1:
        print(f"Step={step} Loss={total_value / total_size}")
        total_value = 0
        total_size = 0

TypeError: ignored